In [1]:
import os, sys
import numpy as np
import pandas as pd

In [2]:
# Get the weather data from all csv files and combine together
weather_data = pd.concat([pd.read_csv("weather/" + file, header=14) for file in os.listdir("Weather")])

# Parse the hour
weather_data = weather_data.rename(columns={'Time': 'Hour'})
weather_data['Hour'] = weather_data['Hour'].apply(lambda x: int(x[:2]))

# Get only the useful columns
weather_data = weather_data[['Year', 'Month', 'Day', 'Hour', 'Weather']].dropna()
'''
weather_data = weather_data.drop([
    'Date/Time',
    'Data Quality',
    'Temp Flag',
    'Dew Point Temp Flag',
    'Rel Hum Flag',
    'Wind Dir Flag',
    'Wind Spd Flag',
    'Visibility Flag',
    'Stn Press Flag',
    'Hmdx',
    'Hmdx Flag',
    'Wind Chill',
    'Wind Chill Flag'
], axis=1)
'''

print(weather_data.shape)
weather_data.head(10)

(5653, 5)


,Year,Month,Day,Hour,Weather
1,2016,5,1,1,Clear
4,2016,5,1,4,Clear
7,2016,5,1,7,Mainly Clear
10,2016,5,1,10,Mainly Clear
13,2016,5,1,13,Mainly Clear
16,2016,5,1,16,Clear
19,2016,5,1,19,Clear
22,2016,5,1,22,Clear
25,2016,5,2,1,Clear
28,2016,5,2,4,Clear


In [3]:
# See what kind of weather we are dealing with
print(weather_data.groupby(['Weather']).agg('count'))

                                              Year  Month   Day  Hour
Weather                                                              
Clear                                          586    586   586   586
Cloudy                                        1181   1181  1181  1181
Drizzle                                         22     22    22    22
Drizzle,Fog                                     18     18    18    18
Fog                                             50     50    50    50
Freezing Fog                                     4      4     4     4
Freezing Rain,Fog                                4      4     4     4
Heavy Rain                                       2      2     2     2
Heavy Rain Showers,Moderate Snow Pellets,Fog     1      1     1     1
Heavy Rain,Fog                                   4      4     4     4
Heavy Rain,Moderate Hail,Fog                     1      1     1     1
Ice Pellets                                      1      1     1     1
Mainly Clear        

In [4]:
# Create two new columns for seperation
weather_data["Visibility"] = "Undefined"
weather_data["Pricipitation"] = "None"

# Change pricipitation column
weather_data.loc[weather_data["Weather"].str.contains("Rain", False), "Pricipitation"] = "Rain"
weather_data.loc[weather_data["Weather"].str.contains("Drizzle", False), "Pricipitation"] = "Rain"
weather_data.loc[weather_data["Weather"].str.contains("Thunderstorms", False), "Pricipitation"] = "Rain"
weather_data.loc[weather_data["Weather"].str.contains("Snow", False), "Pricipitation"] = "Snow"
weather_data.loc[weather_data["Weather"].str.contains("Ice Pellets", False), "Pricipitation"] = "Snow"

# Change visibility column
weather_data.loc[weather_data["Weather"].str.contains("Clear", False), "Visibility"] = "Clear"
weather_data.loc[weather_data["Weather"].str.contains("Cloudy", False), "Visibility"] = "Cloudy"
weather_data.loc[weather_data["Weather"].str.contains("Fog", False), "Visibility"] = "Fog"
print(weather_data.groupby(['Visibility', 'Pricipitation']).agg('count'))

                          Year  Month   Day  Hour  Weather
Visibility Pricipitation                                  
Clear      None           1548   1548  1548  1548     1548
Cloudy     None           2220   2220  2220  2220     2220
Fog        None             54     54    54    54       54
           Rain            207    207   207   207      207
           Snow             14     14    14    14       14
Undefined  Rain           1472   1472  1472  1472     1472
           Snow            138    138   138   138      138


In [5]:
# Make assumptions that visibility is clear when snow, and cloudy when rain
weather_data.loc[(weather_data["Visibility"] == 'Undefined') & (weather_data['Pricipitation'] == 'Snow'), 'Visibility'] = 'Clear'
weather_data.loc[(weather_data["Visibility"] == 'Undefined') & (weather_data['Pricipitation'] == 'Rain'), 'Visibility'] = 'Cloudy'
weather_data = weather_data.drop(['Weather'], axis=1)
weather_data

,Year,Month,Day,Hour,Visibility,Pricipitation
1,2016,5,1,1,Clear,None
4,2016,5,1,4,Clear,None
7,2016,5,1,7,Clear,None
10,2016,5,1,10,Clear,None
13,2016,5,1,13,Clear,None
16,2016,5,1,16,Clear,None
19,2016,5,1,19,Clear,None
22,2016,5,1,22,Clear,None
25,2016,5,2,1,Clear,None
28,2016,5,2,4,Clear,None


In [6]:
# Save the weather
weather_data.to_csv('Weather.csv', index = False,encoding='utf-8')